# Hashing vectorizer 

In [1]:
import re
import hashlib
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

скачайте и распакуйте датасет [Sentiment140 dataset with 1.6 million tweets](https://www.kaggle.com/kazanova/sentiment140) с kaggle

In [2]:
path_to_dataset = "/Users/d.parpulov/Downloads/training.1600000.processed.noemoticon.csv"

In [3]:
dataset = pd.read_csv(path_to_dataset,
                      names = ['sentiment', 'id', 'date', 'flag', 'user', 'text'],
                      header=None, engine='python')
dataset.shape

(1600000, 6)

In [4]:
dataset = dataset[["text", "sentiment"]]
dataset["sentiment"] = dataset["sentiment"].apply(lambda x: 1 if x==4 else x)
dataset.head()

,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [5]:
pd.unique(dataset["sentiment"])

array([0, 1])

Sentiment
- 0 = negative,
- 1 = positive

In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset["text"], dataset["sentiment"], 
                                                    test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape,

((1280000,), (320000,), (1280000,), (320000,))

### Зададим токенизатор
можно использовать и дефолтный, но для наглядности мы используем кастомный и применим его позже для Hashing Vectorizer

In [7]:
def tokenizer(text, token_pattern="(?:\w|\')+", thresh=1):
    text = text.lower()
    text = re.sub("https?:/\S+", '', text)  # remove urls
    tokens = re.findall(token_pattern, text)  # split tokens
    return filter(lambda x: len(x) > thresh, tokens)  # filter short tokens

In [8]:
test_text = dataset["text"].loc[0]
print('text:', test_text, end='\n\n')
print('tokens:', list(tokenizer(test_text)))

text: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D

tokens: ['switchfoot', 'awww', "that's", 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it']


### Используем обычный TfidfVectorizer с нашим токенизатором и дефолтными параметрами

In [9]:
%%time
vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

CPU times: user 25.8 s, sys: 314 ms, total: 26.1 s
Wall time: 26.2 s


In [10]:
X_train_vect.shape, X_test_vect.shape  # 560k признаков! 

((1280000, 560343), (320000, 560343))

### Обучим SVM

In [11]:
clf = LinearSVC(penalty="l2")

In [12]:
%%time
clf.fit(X_train_vect, y_train)
predicted = clf.predict(X_test_vect)

print("Precision: %.3f" % precision_score(y_test, predicted))
print("Recall: %.3f" % recall_score(y_test, predicted))
print("F1: %.3f" % f1_score(y_test, predicted))

Precision: 0.795
Recall: 0.802
F1: 0.798
CPU times: user 27.2 s, sys: 319 ms, total: 27.5 s
Wall time: 29.1 s


Полный набор из 560к признаков дает качество из коробки F1=0.798

## Похешируем фичи

In [12]:
def hasher(token):
    hashed_feature = int(hashlib.sha1(token.encode('utf-8')).hexdigest(), 16)
    return hashed_feature

In [8]:
s = "switchfoot"  # исходный токен
hasher(s)  # хешированный токен

493129059604241748898866719106155948524683866694

Хеши длинные и получаются уникальными для каждого токена. 

Чтобы сделать набор новых признаков поменьше, уменьшим количество бакетов

In [14]:
def hasher(token, n_buckets=10000):
    hashed_feature = int(hashlib.sha1(token.encode('utf-8')).hexdigest(), 16)
    hashed_feature %= n_buckets
    return hashed_feature

In [10]:
s = "switchfoot"  # исходный токен
hasher(s)  # хешированный токен, лежит в диапазоне 0-10k

6694

In [17]:
def hashed_tokenizer(text, n_buckets=10000, token_pattern='\w+', thresh=2):
    text = text.lower()
    text = re.sub("https?:/\S+", '', text)  # remove urls
    tokens = re.findall(token_pattern, text)  # split tokens
    filtered_tokens = filter(lambda x: len(x) > thresh, tokens)  # filter short tokens    
    hashed_features = map(lambda x: hasher(x, n_buckets), filtered_tokens)
    return hashed_features

In [18]:
print('text:', test_text, end='\n'*2)
print('tokens:', list(tokenizer(test_text)), end='\n'*2)
print('hashed tokens:', list(hashed_tokenizer(test_text)))

text: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D

tokens: ['switchfoot', 'awww', "that's", 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it']

hashed tokens: [6694, 9791, 9451, 8546, 4642, 7894, 3896, 4388, 9566, 7129, 9991]


## Построим TfIdfVectorizer над хешированными фичами
Это как раз то, что делает под капотом sklearn.HashingVectorizer

In [19]:
%%time
hashing_vectorizer = TfidfVectorizer(tokenizer=hashed_tokenizer)
X_train_hashvect = hashing_vectorizer.fit_transform(X_train)
X_test_hashvect = hashing_vectorizer.transform(X_test)

CPU times: user 47.7 s, sys: 455 ms, total: 48.2 s
Wall time: 49.5 s


In [20]:
X_train_hashvect.shape, X_test_hashvect.shape  # 10k новых признаков 

((1280000, 10000), (320000, 10000))

In [21]:
%%time
clf.fit(X_train_hashvect, y_train)
predicted = clf.predict(X_test_hashvect)

print("Precision: %.3f" % precision_score(y_test, predicted))
print("Recall: %.3f" % recall_score(y_test, predicted))
print("F1: %.3f" % f1_score(y_test, predicted))

Precision: 0.762
Recall: 0.785
F1: 0.773
CPU times: user 27.5 s, sys: 324 ms, total: 27.8 s
Wall time: 29.4 s


F1=0.773 для 10к признаков

F1=0.798 - для 560к признаков

Не особо заморачиваясь, снизили количество фичей в 50+ раз, и потеряли в качестве всего около ~3%

## А теперь проверим, как работает родной HashingVectorizer из sklearn

In [22]:
%%time
hashing_vectorizer_sklearn = HashingVectorizer(n_features=10000)
X_train_sklearn = hashing_vectorizer_sklearn.fit_transform(X_train)
X_test_sklearn = hashing_vectorizer_sklearn.transform(X_test)

CPU times: user 13.4 s, sys: 161 ms, total: 13.6 s
Wall time: 13.8 s


заметно быстрее, чем наш самопальный

In [23]:
%%time
clf.fit(X_train_sklearn, y_train)
predicted = clf.predict(X_test_sklearn)

print("Precision: %.3f" % precision_score(y_test, predicted))
print("Recall: %.3f" % recall_score(y_test, predicted))
print("F1: %.3f" % f1_score(y_test, predicted))

Precision: 0.771
Recall: 0.783
F1: 0.777
CPU times: user 28.3 s, sys: 287 ms, total: 28.6 s
Wall time: 29.5 s


и чуть лучше по качеству

# Fasttext

инструкции для установки фасттекста [тут](https://github.com/facebookresearch/fastText/tree/master/python)

In [75]:
import fasttext

from sklearn.metrics.pairwise import cosine_similarity

Используем тот же датасет, что и ранее

In [25]:
# подготовим файлы с датасетами для фасттекста
dataset["label_ft"] = dataset["sentiment"].apply(lambda x: "__label__" + str(x))

In [26]:
train_dataset = pd.concat([X_train.apply(lambda x: re.sub('\t', '', x)), 
                           y_train.apply(lambda x: '__label__' + str(x))], axis=1)

test_dataset = pd.concat([X_test.apply(lambda x: re.sub('\t', '', x)), 
                          y_test.apply(lambda x: '__label__' + str(x))], axis=1)

сохраним в формате 'текст \t метка'

In [27]:
train_dataset.to_csv("train_fasttext.txt", sep='\t', header=None, index=None)
test_dataset.to_csv("test_fasttext.txt", sep='\t', header=None, index=None)

In [28]:
! head -2 train_fasttext.txt

@jbtaylor WIth ya. &quot;I'd like a Palm Pre, Touchstone charger. ReadyNow? Yes, that sounds good. But is my beer ready now?'  #prelaunch	__label__1
felt the earthquake this afternoon, it seems to be a , but  at the epicenter 	__label__1


In [29]:
!cut -f2 train_fasttext.txt | sort | uniq -c

640506 __label__0
639494 __label__1


### Запустим обучение фасттекста на необработанном датасете

In [38]:
def print_results(N, p, r):
    print("Precision\t{:.3f}".format(p))
    print("Recall\t{:.3f}".format(r))
    print("F1\t{:.3f}".format(2*p*r/(p+r)))

In [99]:
%%time
model_big = fasttext.train_supervised(
    input="train_fasttext.txt",
    minCount=5,  # отсеиваем редкие токены
    minn=3, maxn=5,  # диапазон для символьных нграмм
    wordNgrams=2,  # используем словесные нграммы размера 2
    dim=25  # размер вектора
) # логи обучения пишутся в терминале

CPU times: user 1min 56s, sys: 1.32 s, total: 1min 57s
Wall time: 45.9 s


In [100]:
model_big.save_model("model_big.ft")

In [102]:
ls -lh | grep model_big.ft  # 203 Mб моделька - не слабо!

-rw-r--r--  1 d.parpulov  staff   203M 29 ноя 11:16 model_big.ft


In [104]:
print_results(*model_big.test('test_fasttext.txt'))

Precision	0.819
Recall	0.819
F1	0.819


и сразу из коробки получили качество, лучше чем до этого на 2-3%

Но модель весит многовато. Давайте попробуем ужать количество бакетов для хеширования нграмм

In [115]:
%%time
model_small = fasttext.train_supervised(
    input="train_fasttext.txt",
    minCount=5,  # отсеиваем редкие токены
    minn=3, maxn=5,  # диапазон для символьных нграмм
    wordNgrams=2,  # используем словесные нграммы размеры 2
    dim=25, # размер вектора
    bucket=200000, # количество бакетов для хеширования
) # логи обучения пишутся в терминале

CPU times: user 1min 31s, sys: 1.05 s, total: 1min 32s
Wall time: 35.2 s


In [116]:
model_small.save_model("model_smaller.ft")

In [1]:
ls -lh | grep model_smaller  # 32M моделька - уже лучше

-rw-r--r--  1 d.parpulov  staff    32M 29 ноя 11:21 model_smaller.ft


In [118]:
print_results(*model_small.test('test_fasttext.txt'))

Precision	0.813
Recall	0.813
F1	0.813


Видим, что модель стала сильно меньше, но а качество просело на полпроцента

У фасттекста есть замечательная фича квантизации, которая позволяет сжимать модели с минимальной потерей качества

Проверим, как она будет работать на меньшей модели

In [119]:
model_small.quantize(input="train_fasttext.txt", retrain=True)

In [120]:
model_small.save_model("model_compressed.ftz")

In [121]:
ls -lh | grep model_compressed  # итоговая модель весит всего 5,8 Мб

-rw-r--r--  1 d.parpulov  staff   5,8M 29 ноя 11:22 model_compressed.ftz


In [122]:
print_results(*model_small.test('test_fasttext.txt'))  # а качество показывает такое же, как и непожатая модель

Precision	0.813
Recall	0.813
F1	0.813


### Посмотрим методы модели

In [123]:
# предскажем метку для текста
model_small.predict("i hate everything about you!")

(('__label__0',), array([0.81586701]))

In [124]:
# получим вектор слова
model_small.get_word_vector('man')

array([-0.00155861,  0.00076249, -0.00773714, -0.00307127,  0.00364631,
       -0.00418105,  0.00483813, -0.00204994,  0.00718853,  0.01275729,
        0.00109148,  0.00380854, -0.01695282, -0.00075939, -0.00462876,
        0.00279449,  0.01129375,  0.00941746,  0.01609236,  0.0181504 ,
       -0.02060649, -0.01163181, -0.01096021, -0.00530058, -0.00763473],
      dtype=float32)

In [125]:
# получим вектор предложения
model_small.get_sentence_vector("i hate everything about you!")

array([ 2.3495326e-05,  3.3990247e-03, -5.5876030e-03, -3.6635119e-03,
        3.6372214e-03, -1.6306896e-03, -9.2436257e-04, -1.6920323e-03,
        1.0075700e-02,  9.9754510e-03,  1.7363178e-03,  1.0817442e-03,
       -1.3282857e-02,  3.0627372e-03, -3.2718773e-03,  4.3372135e-03,
        4.2888047e-03,  4.3199151e-03,  1.3004756e-02,  1.6394349e-02,
       -8.6986041e-03, -3.5347007e-03, -4.7837254e-03, -4.4796113e-03,
       -2.4813798e-03], dtype=float32)

In [127]:
# проверим, насколько похожи по контенту два предложения на основе косинусного расстояния между эмбедингами 
a = model_small.get_sentence_vector("I feel lousy")
b = model_small.get_sentence_vector("i hate everything about you!")
cosine_similarity([a],[b])

array([[0.97241944]], dtype=float32)

In [128]:
a = model_small.get_sentence_vector("i like that awesome song!")
b = model_small.get_sentence_vector("i hate everything about you!")
cosine_similarity([a],[b])

array([[-0.97110647]], dtype=float32)